# **Import Library**

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import plotly.express as px
import glob
from scipy import stats

print('numpy version : ',np.__version__)
print('pandas version : ',pd.__version__)
print('seaborn version : ',sns.__version__)

numpy version :  1.26.4
pandas version :  2.2.0
seaborn version :  0.13.2


In [2]:
from matplotlib import rcParams

rcParams['figure.figsize'] = 12, 4
rcParams['lines.linewidth'] = 3
rcParams['xtick.labelsize'] = 'large'
rcParams['ytick.labelsize'] = 'large'

# **Import Dataset**

In [4]:
df = pd.read_csv('train_prototipe.csv')
df.sample(10)

,Calls,Cost,Purchases,Importance,Discount,Weight (gram),Late,Weight_category_encoded,Reorder_purchases
38,0.5,0.012195,1.0,0,8.166667,-0.076001,0.0,0.000000,0.0
4356,0.0,-0.853659,0.0,0,-1.000000,0.539480,-1.0,1.333333,0.0
10901,0.0,-0.158537,-1.0,1,0.000000,0.053263,-1.0,0.666667,-1.0
10587,-0.5,-0.585366,0.0,1,0.333333,0.350724,0.0,1.333333,0.0
9538,-0.5,-0.024390,0.0,1,0.500000,-0.009967,-1.0,0.000000,0.0
7426,-1.0,-1.024390,1.0,1,0.500000,0.066656,-1.0,0.666667,0.0
9372,0.0,0.695122,1.0,1,0.333333,0.282822,-1.0,1.333333,0.0
9563,0.0,-0.280488,-1.0,0,0.500000,0.286248,0.0,1.333333,-1.0
10910,0.5,0.609756,5.0,2,0.500000,0.501480,0.0,1.333333,0.0
9909,1.0,0.378049,1.0,0,0.333333,-0.795826,-1.0,-0.666667,0.0


# **Stage 3**